In [ ]:
import pandas as pd
import catboost as cb

from catboost import Pool, CatBoostClassifier
from sklearn.metrics import f1_score, accuracy_score, classification_report

from typing import List

random_state = 42

## Data Processing

In [ ]:
# columns
target_gender = "sex"
target_age = "age_class"

cat_features = ["region", "ua_device_type", "ua_client_type", "ua_os", "ua_client_name", "category"]
text_feature = "title"
date_feature = "event_timestamp"

drop_feature = "age"

id_columns = ["viewer_uid", "rutube_video_id", "author_id"]

In [ ]:
dataset = pd.read_csv('TEST-DATA').drop(
    drop_feature, axis=1
).fillna('none')

# new feature
dataset['videos_per_day'] = dataset.groupby(['viewer_uid', 'day']).transform('size')

# drop unimportant old features
dataset = dataset.drop(["second", "minute", "month","year"], axis=1)

## Utils

In [ ]:
def set_pool(data: pd.DataFrame, target: str, id_columns: List[str]) -> cb.core.Pool:
    """
    Prepares the data as a CatBoost Pool object, separating features, target, and categorical/text features.
    """
    return Pool(
        data=data.drop(id_columns + [target], axis=1),
        cat_features=cat_features,
        text_features=[text_feature],
        label=data[target]
    )


def print_classification_result(real: pd.Series, pred: pd.Series, multiclass: bool = True):
    """
    Prints the classification metrics (Accuracy or F1-score) and detailed classification report.
    """
    if not multiclass: 
        result_score = f'Accuracy: {accuracy_score(real, pred)}'
    else:
        result_score = f'F1: {f1_score(real, pred, average="weighted")}'
    
    print(
        result_score,
        classification_report(real, pred),
        sep='\n'
    )
    
    
def aggregate_score_by_user(test: pd.DataFrame, preds: pd.DataFrame):
    """
    Aggregates predictions by user ID by taking the mode of the predicted values.

    This function adds the predictions to the test DataFrame and then computes 
    the most common prediction (mode) for each user based on their viewer_uid.
    """
    test_with_preds = test
    test_with_preds['preds'] = preds

    value_counts_preds = test_with_preds.groupby('viewer_uid')['preds'].apply(lambda x: x.mode()[0])
    value_counts_preds = value_counts_preds.to_dict()
    final_preds = test['viewer_uid'].apply(lambda x: value_counts_preds[x])
    return final_preds


def final_score(
    gender_real: pd.DataFrame,
    gender_pred: pd.DataFrame, 
    age_real: pd.DataFrame, 
    age_pred: pd.DataFrame
):
    """
    Computes the final score as a weighted combination of gender accuracy and age F1-score.
    """
    gender = accuracy_score(gender_real, gender_pred)
    age = f1_score(age_real, age_pred, average="weighted")
    result = 0.3 * gender + 0.7 * age
    print(f'Final score: {result}')
    return result

## Predict

In [ ]:
# age model
age_test_pool = set_pool(dataset, target_age, id_columns)

age_model = CatBoostClassifier().load_model('MODEL')

In [ ]:
# gender model 
gender_test_pool = set_pool(dataset, target_gender, id_columns)

gender_model = CatBoostClassifier().load_model('MODEL')

In [ ]:
age_predict = age_model.predict(age_test_pool)
gender_predict = gender_model.predict(gender_test_pool)

In [ ]:
final_age_predict = aggregate_score_by_user(dataset, age_predict)
final_gender_predict = aggregate_score_by_user(dataset, gender_predict)

In [ ]:
print_classification_result(dataset[target_age], age_predict)

In [ ]:
print_classification_result(dataset[target_gender], gender_predict)

In [ ]:
final_score(
    dataset[gender_predict],
    gender_predict,
    dataset[age_predict],
    age_predict
)